In [14]:
!pip install transformers
!pip install datasets
from datasets import load_dataset, concatenate_datasets
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
# Italian model
tokenizer = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-sentiment")
model_it = AutoModelForSequenceClassification.from_pretrained("MilaNLProc/feel-it-italian-sentiment")
# Load the dataset
dataset = load_dataset("sepidmnorozy/Maltese_sentiment")
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]
test_dataset = dataset["test"]
# Concatenate the splits into one dataset
maltese_data = concatenate_datasets([train_dataset, validation_dataset, test_dataset])
df = pd.DataFrame(maltese_data)

# Function to perform sentiment analysis
def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model_it(**inputs)
    prediction_index = torch.argmax(outputs.logits, dim=1).item()  # Index of the highest logit score
    return prediction_index

df['predicted_sentiment'] = df['text'].apply(sentiment_analysis)
print(df)

# Actual VS predicted
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df['label'], df['predicted_sentiment'])
print(f"Accuracy: {accuracy}")


     label                                               text  \
0        1  Nemmen li huwa baġit tajjeb, imma nies bħali t...   
1        0  Nippremjaw lil min ilu jikser il-liġi snin u j...   
2        0  Xi cuc irrid jitfacca b'kritika ta' din ix-xorta.   
3        0  Ms. Metzola, ara jekk Ms. Mizzi jkollhiex ċans...   
4        1  Naf ċert li int qegħda hemm fejn il-Mulej tkom...   
..     ...                                                ...   
846      0  Dil-bużżieqa li nefħu qed tisplodi f'wiċċhom m...   
847      0  Dan imissu jintbagħat lura minn fejn ġie u ma'...   
848      0                   Il-moda tikrih maż-żmien qegħda.   
849      0  Aħna familja Laburista kbira u ngħidlek li taħ...   
850      0  Tajjeb jidher fuq il-karta. Imma l-fatti jidhr...   

     predicted_sentiment  
0                      1  
1                      0  
2                      0  
3                      1  
4                      1  
..                   ...  
846                    1  
847

In [18]:
# Confusion matrix calculation
tn, fp, fn, tp = confusion_matrix(df['label'], df['predicted_sentiment']).ravel()

# Printing the counts
print(f"Actual Positive, Predicted Positive: {tp}")
print(f"Actual Positive, Predicted Negative: {fn}")
print(f"Actual Negative, Predicted Positive: {fp}")
print(f"Actual Negative, Predicted Negative: {tn}")

Actual Positive, Predicted Positive: 244
Actual Positive, Predicted Negative: 27
Actual Negative, Predicted Positive: 460
Actual Negative, Predicted Negative: 120


In [19]:
df['actual_label'] = df['label']  # Replace 'label' with the actual column name for labels in your DataFrame

# Filter for each category
true_positives = df[(df['actual_label'] == 1) & (df['predicted_sentiment'] == 1)]
true_negatives = df[(df['actual_label'] == 0) & (df['predicted_sentiment'] == 0)]
false_positives = df[(df['actual_label'] == 0) & (df['predicted_sentiment'] == 1)]
false_negatives = df[(df['actual_label'] == 1) & (df['predicted_sentiment'] == 0)]

# Print 10 examples from each
print("True Positives (10 examples):")
print(true_positives.head(10), '\n')

print("True Negatives (10 examples):")
print(true_negatives.head(10), '\n')

print("False Positives (10 examples):")
print(false_positives.head(10), '\n')

print("False Negatives (10 examples):")
print(false_negatives.head(10), '\n')

True Positives (10 examples):
    label                                               text  \
0       1  Nemmen li huwa baġit tajjeb, imma nies bħali t...   
4       1  Naf ċert li int qegħda hemm fejn il-Mulej tkom...   
6       1  Sewwa jgħid il-Malti - Kelb rieqed la tqajmhux...   
8       1  Malta għandha xorti li għandna lil Alfred Sant...   
10      1  Grazzi ta' kemm ferraħtna tul dawn il-ħamsin s...   
14      1  Prosit tassew lil kull min ħa sehem f'din il-b...   
26      1  Tajjeb li n-nies jagħmlu użu aħjar mit-transpo...   
31      1  Artiklu miktub bis-sens, u sa ċertu sens apoli...   
32      1  Nifraħhielha tas-suċċess kemm tal-Fun Run li k...   
41      1  Il-ġenituri fil-maġġoranza tagħhom iridu lill-...   

    predicted_sentiment  actual_label  
0                     1             1  
4                     1             1  
6                     1             1  
8                     1             1  
10                    1             1  
14                    1  